In [1]:
!pip install transformers datasets evaluate pandas torch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 50.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 49.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling 

In [2]:
import pandas as pd
import torch
from datasets import Dataset
from transformers import AutoTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from transformers import DataCollatorWithPadding
from evaluate import load
import re


In [3]:
stopwords = [
    "I", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your",
    "yours", "yourself", "yourselves", "he", "him", "his", "himself", "she",
    "her", "hers", "herself", "it", "its", "itself", "they", "them", "their",
    "theirs", "themselves", "what", "which", "who", "whom", "this", "that",
    "these", "those", "am", "is", "are", "was", "were", "be", "been", "being",
    "have", "has", "had", "having", "do", "does", "did", "doing", "a", "an",
    "the", "and", "but", "if", "or", "because", "as", "until", "while", "of",
    "at", "by", "for", "with", "about", "against", "between", "into", "through",
    "during", "before", "after", "above", "below", "to", "from", "up", "down",
    "in", "out", "on", "off", "over", "under", "again", "further", "then", "once",
    "here", "there", "when", "where", "why", "how", "all", "any", "both", "each",
    "few", "more", "most", "other", "some", "such", "no", "nor", "not", "only",
    "own", "same", "so", "than", "too", "very", "s", "t", "can", "will", "just",
    "don", "should", "now"
]

In [4]:
emoji_dict = {
    "😊": "Happy",
    "😢": "Sad",
    "😠": "Angry",
    "😔": "Upset",
    "👍": "Great",
    "😎": "Cool",
    "😭": "Crying",
    "😁": "Grinning",
    "😅": "Slightly Smiling",
    "😍": "Love",
    "😒": "Unamused",
    "😞": "Disappointed",
    "😡": "Furious",
    "😃": "Cheerful",
    "😉": "Winking",
    "😋": "Tasty",
    "😐": "Neutral",
    "😤": "Frustrated",
    "😴": "Sleepy",
    "😜": "Playful",
    "😩": "Exhausted",
    "😯": "Surprised",
    "😆": "Laughing",
    "😷": "Sick",
    "🙄": "Eye Rolling",
    "😳": "Shocked",
    "😬": "Nervous",
    "😚": "Kissing",
    "😰": "Anxious",
    "🤗": "Hugging",
    "🤔": "Thinking",
    "🤐": "Silent",
    "😇": "Innocent"
}


In [5]:
def preprocess_text(text):
    if pd.isnull(text) or not isinstance(text, str):
        return None

    # Convert text to lowercase
    text = text.lower()

    # Remove URLs
    text = re.sub(r"http\S+|www\S+|https\S+", "", text, flags=re.MULTILINE)

    # Remove special characters, numbers, and punctuations (except spaces and alphabets)
    text = re.sub(r"[^a-zA-Z\s]", "", text)

    # Replace emojis with corresponding text
    for emoji, meaning in emoji_dict.items():
        text = text.replace(emoji, meaning)

    # Remove stopwords
    words = text.split()
    words = [word for word in words if word not in stopwords]

    # Remove any extra spaces
    text = " ".join(words)

    return text.strip() if text else None

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/NLP/train.csv",encoding="ISO-8859-1")
#encoding="ISO-8859-1

In [ ]:
df.head(15)

,textID,text,selected_text,sentiment,Time of Tweet,Age of User,Country,Population -2020,Land Area (Km²),Density (P/Km²)
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral,morning,0-20,Afghanistan,38928346,652860.0,60
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,noon,21-30,Albania,2877797,27400.0,105
2,088c60f138,my boss is bullying me...,bullying me,negative,night,31-45,Algeria,43851044,2381740.0,18
3,9642c003ef,what interview! leave me alone,leave me alone,negative,morning,46-60,Andorra,77265,470.0,164
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative,noon,60-70,Angola,32866272,1246700.0,26
5,28b57f3990,http://www.dothebouncy.com/smf - some shameles...,http://www.dothebouncy.com/smf - some shameles...,neutral,night,70-100,Antigua and Barbuda,97929,440.0,223
6,6e0c6d75b1,2am feedings for the baby are fun when he is a...,fun,positive,morning,0-20,Argentina,45195774,2736690.0,17
7,50e14c0bb8,Soooo high,Soooo high,neutral,noon,21-30,Armenia,2963243,28470.0,104
8,e050245fbd,Both of you,Both of you,neutral,night,31-45,Australia,25499884,7682300.0,3
9,fc2cbefa9d,Journey!? Wow... u just became cooler. hehe....,Wow... u just became cooler.,positive,morning,46-60,Austria,9006398,82400.0,109


In [ ]:
df.shape

(27481, 10)

In [ ]:
df.columns

Index(['textID', 'text', 'selected_text', 'sentiment', 'Time of Tweet',
       'Age of User', 'Country', 'Population -2020', 'Land Area (Km²)',
       'Density (P/Km²)'],
      dtype='object')

In [ ]:
df_selected = df[['selected_text', 'sentiment']]

In [ ]:
df_selected.head()

,selected_text,sentiment
0,"I`d have responded, if I were going",neutral
1,Sooo SAD,negative
2,bullying me,negative
3,leave me alone,negative
4,"Sons of ****,",negative


In [ ]:
df=df_selected

In [ ]:
df.head()

,selected_text,sentiment
0,"I`d have responded, if I were going",neutral
1,Sooo SAD,negative
2,bullying me,negative
3,leave me alone,negative
4,"Sons of ****,",negative


In [ ]:
df['text']=df['selected_text']

In [ ]:
df['label']=df['sentiment']

In [ ]:
df=df.drop('selected_text',axis=1)

In [ ]:
df=df.drop('sentiment',axis=1)

In [ ]:
df.head()

,text,label
0,"I`d have responded, if I were going",neutral
1,Sooo SAD,negative
2,bullying me,negative
3,leave me alone,negative
4,"Sons of ****,",negative


In [ ]:
df['label'].unique()

array(['neutral', 'negative', 'positive'], dtype=object)

In [ ]:
df.isnull().sum()

,0
text,1
label,0


In [ ]:
df['label']=df['label'].str.strip()

In [ ]:
df['label'].unique()

array(['neutral', 'negative', 'positive'], dtype=object)

In [ ]:
df.head()

,text,label
0,"I`d have responded, if I were going",neutral
1,Sooo SAD,negative
2,bullying me,negative
3,leave me alone,negative
4,"Sons of ****,",negative


In [ ]:
df = df.reset_index(drop=True)

In [ ]:
df.head()

,text,label
0,"I`d have responded, if I were going",neutral
1,Sooo SAD,negative
2,bullying me,negative
3,leave me alone,negative
4,"Sons of ****,",negative


In [ ]:
# Load dataset
#df = pd.read_csv("/content/drive/MyDrive/NLP/bangla_sentiment_data.csv")

# Map labels to integers
label_mapping = {"positive": 0, "negative": 1, "neutral": 2}

# Apply preprocessing to the text and drop invalid rows
df["text"] = df["text"].apply(preprocess_text)
df = df.dropna(subset=["text"])  # Drop rows with None text
df["label"] = df["label"].map(label_mapping)


In [ ]:
df.head()

,text,label
0,id responded i going,2
1,sooo sad,1
2,bullying,1
3,leave alone,1
4,sons,1


In [ ]:
df['label'].unique()

array([2, 1, 0])

In [ ]:
df.shape

(27291, 2)

In [ ]:
# Convert to Hugging Face Dataset
hf_dataset = Dataset.from_pandas(df)


In [ ]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

# Tokenization function
def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        padding="max_length",
        truncation=True,
        max_length=128  # Set max_length for consistency
    )

# Tokenize the dataset
tokenized_dataset = hf_dataset.map(tokenize_function, batched=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/27291 [00:00<?, ? examples/s]

In [ ]:
# Split dataset into train and validation sets
train_test_split = tokenized_dataset.train_test_split(test_size=0.2)
train_dataset = train_test_split["train"]
valid_dataset = train_test_split["test"]


In [ ]:
# Data collator to handle dynamic padding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Load Bangla BERT model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",  # Updated key
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_dir="./logs",
    save_strategy="epoch",
    report_to="none",  # Disable all integrations like W&B
)


In [ ]:
!pip install wandb

In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"


In [ ]:
# Load accuracy metric from `evaluate`
metric = load("accuracy")

# Define compute metrics function
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), axis=1)
    return metric.compute(predictions=predictions.numpy(), references=labels)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    compute_metrics=compute_metrics,
    data_collator=data_collator,
)


In [ ]:
# Train the model
trainer.train()

# Evaluate the model
metrics = trainer.evaluate()
print("Evaluation Metrics:", metrics)


Epoch,Training Loss,Validation Loss,Accuracy
1,0.439500,0.427095,0.862979
2,0.345100,0.500492,0.859864
3,0.244800,0.655568,0.861330
4,0.190100,0.746781,0.859864
5,0.120400,0.801572,0.857483


Evaluation Metrics: {'eval_loss': 0.8015719652175903, 'eval_accuracy': 0.8574830555046712, 'eval_runtime': 38.1957, 'eval_samples_per_second': 142.922, 'eval_steps_per_second': 17.882, 'epoch': 5.0}


In [ ]:
# Save the fine-tuned model
trainer.save_model("/content/drive/MyDrive/NLP/english_bert_sentiment_model")
tokenizer.save_pretrained("/content/drive/MyDrive/NLP/english_bert_sentiment_model")


('/content/drive/MyDrive/NLP/english_bert_sentiment_model/tokenizer_config.json',
 '/content/drive/MyDrive/NLP/english_bert_sentiment_model/special_tokens_map.json',
 '/content/drive/MyDrive/NLP/english_bert_sentiment_model/vocab.txt',
 '/content/drive/MyDrive/NLP/english_bert_sentiment_model/added_tokens.json',
 '/content/drive/MyDrive/NLP/english_bert_sentiment_model/tokenizer.json')

In [6]:
import torch
import re
import pandas as pd
from transformers import AutoTokenizer, BertForSequenceClassification

# Load the fine-tuned model and tokenizer
model_path = "/content/drive/MyDrive/NLP/english_bert_sentiment_model"
model = BertForSequenceClassification.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Move model to appropriate device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Corrected label mapping (number to sentiment)
label_mapping = {0: "positive", 1: "negative", 2: "neutral"}

# Define a function to preprocess the input text
def preprocess_text(text):
    stopwords = [
        "I", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your",
        "yours", "yourself", "yourselves", "he", "him", "his", "himself", "she",
        "her", "hers", "herself", "it", "its", "itself", "they", "them", "their",
        "theirs", "themselves", "what", "which", "who", "whom", "this", "that",
        "these", "those", "am", "is", "are", "was", "were", "be", "been", "being",
        "have", "has", "had", "having", "do", "does", "did", "doing", "a", "an",
        "the", "and", "but", "if", "or", "because", "as", "until", "while", "of",
        "at", "by", "for", "with", "about", "against", "between", "into", "through",
        "during", "before", "after", "above", "below", "to", "from", "up", "down",
        "in", "out", "on", "off", "over", "under", "again", "further", "then", "once",
        "here", "there", "when", "where", "why", "how", "all", "any", "both", "each",
        "few", "more", "most", "other", "some", "such", "no", "nor", "not", "only",
        "own", "same", "so", "than", "too", "very", "s", "t", "can", "will", "just",
        "don", "should", "now"
    ]

    # Define an emoji dictionary
    emoji_dict = {
        "😊": "Happy", "😢": "Sad", "😠": "Angry", "😔": "Upset", "👍": "Great",
        "😎": "Cool", "😭": "Crying", "😁": "Grinning", "😅": "Slightly Smiling",
        "😍": "Love", "😒": "Unamused", "😞": "Disappointed", "😡": "Furious",
        "😃": "Cheerful", "😉": "Winking", "😋": "Tasty", "😐": "Neutral",
        "😤": "Frustrated", "😴": "Sleepy", "😜": "Playful", "😩": "Exhausted",
        "😯": "Surprised", "😆": "Laughing", "😷": "Sick", "🙄": "Eye Rolling",
        "😳": "Shocked", "😬": "Nervous", "😚": "Kissing", "😰": "Anxious",
        "🤗": "Hugging", "🤔": "Thinking", "🤐": "Silent", "😇": "Innocent"
    }

    if pd.isnull(text) or not isinstance(text, str):
        return None

    # Convert text to lowercase
    text = text.lower()

    # Remove URLs
    text = re.sub(r"http\S+|www\S+|https\S+", "", text, flags=re.MULTILINE)

    # Replace emojis with corresponding text
    for emoji, meaning in emoji_dict.items():
        text = text.replace(emoji, meaning)

    # Remove special characters, numbers, and punctuations (except spaces and alphabets)
    text = re.sub(r"[^a-zA-Z\s]", "", text)

    # Remove stopwords
    words = text.split()
    words = [word for word in words if word not in stopwords]

    # Remove extra spaces
    text = " ".join(words)

    return text.strip() if text else None

# Define a function to predict sentiment
def predict_sentiment(text):
    # Preprocess the text
    text = preprocess_text(text)
    if not text:
        return "Invalid input"

    # Tokenize the text
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=128)

    # Move inputs to the same device as the model
    inputs = {key: value.to(device) for key, value in inputs.items()}

    # Predict sentiment
    with torch.no_grad():
        model.eval()
        outputs = model(**inputs)
        logits = outputs.logits
        prediction = torch.argmax(logits, dim=1).item()

    # Return the predicted sentiment
    return label_mapping.get(prediction, "Unknown")



In [7]:
# Example usage
input_text = "you are the fool"
sentiment = predict_sentiment(input_text)
print(f"Sentiment: {sentiment}")


Sentiment: negative


In [9]:
# Example usage
input_text =str(input("Enter the text: "))
sentiment = predict_sentiment(input_text)
print(f"Sentiment: {sentiment}")


Enter the textSons of ****
Sentiment: negative
